# Project Planning Stage (Group)

## **Introduction**

Background Recruiting and retaining active users is critical for running online gaming experiments. The UBC Minecraft research server, led by Prof. Frank Wood, offers an open-access environment in which every movement and click are automatically logged. While the server generates rich behavioural data, only a small fraction of players eventually subscribe to the project’s game-related newsletter—a low-cost channel for announcing new experiments, updates, and funding opportunities. Being able to predict subscription likelihood from readily available player attributes would allow the team to focus recruitment messages on users most likely to engage, thereby increasing subscription rates without expanding marketing effort.

Primary Question Can a player’s experience level, playing time, and age predict whether a player subscribes to the game-related newsletter? Datasets One CSV file is provided and is linkable via the primary key player_id.

players.csv – one row per unique participant. After importing, the merged file contains 1,842 players. The data is collected between 2024-08-01 and 2024-11-30.

*Edited intro if you want to change it*

Recruiting and retaining active users is critical for running online gaming experiments. The UBC Minecraft research server logs all player activity in detail. However, only a small fraction of players subscribe to the game-related newsletter. Because the newsletter is used to announce experiments, updates, and funding opportunities, understanding what influences subscription behavior can help the team target their messages in a more effective way.

For this project, our goal is to explore the question:
*"Can a player's experience level, playing time, and age predict whether a player subscribes to the game-related newsletter?"*

We will be using the players.csv dataset that contains one row per user and other information like experience level, total hours played, age, and subscription status. The data is collected between 2024-08-01 and 2024-11-30.


**Describe why we used said predictors and why we didn't use other predictors**

**Descriptive summary of dataset:**
- The players dataset contains 196 observations and 7 variables.

  Below is a summary of the variables used in our analysis:

| Variable Name | Data Type   | Description / Meaning                                                                                       | Notes / Potential Issues                                                                                |
| ------------- | ----------- | ----------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------- |
| Experience    | Categorical | Player’s self-reported experience level (Beginner, Amateur, Regular, Veteran, Pro)                          | Reflects player skill/familiarity                                                                       |
| Subscribe     | Logical     | Whether players are subscribed to the newsletter (TRUE/FALSE)                                               | May be influenced by external sources other than gameplay experience (marketing, interest)              |
| hashedEmail   | Categorical | Unique hashed player identifier (anonymized)                                                                | Used to identify players; no direct analytical value                                                    |
| played_hours  | Numeric     | Total hours played on server                                                                                | Includes 0 hours (inactive or new players); possible outliers with high values                          |
| name          | Categorical | Player’s first name                                                                                         | Nominal data with potential duplicates                                                                  |
| gender        | Categorical | Player’s self-identified gender (Male, Female, Non-binary, Two-Spirited, Agender, Other, Prefer not to say) | Multiple categories with social diversity; minority groups may require special attention/representation |
| age           | Integer     | Player’s age in years                                                                                       | Large range of ages; two missing data points                                                            |

- Summary statistics table for numeric/integer variables:
  | Variable     | Min  | Mean  | Median | Max   | Std Dev |
| ------------ | ---- | ----- | ------ | ----- | ------- |
| played_hours | 0.00 | 5.85  | 0.1   | 223.10 | 28.36   |
| age          | 9.00 | 21.14 | 19.00  | 58.00 | 7.39    |

## **Methods and Results**

Below, we created a cleaned version of the dataset by selecting some of the previously mentioned variables and removing rows with missing values.

In [ ]:
install.packages("tidyverse")

library(tidyverse)
library(knitr)
library(GGally)
library(ggplot2)
library(dplyr)
library(tidymodels)
library(tidyclust)
library(themis)
library(janitor)
library(rsample)
library(repr)
options(repr.matrix.max.rows = 6)
     

In [ ]:
players <- read_csv("players.csv")

head(players)
glimpse(players)

In [ ]:
#Summary Statistics

players_summary <- players |>
  summarise(
    across(where(is.numeric),
           list(min = ~round(min(., na.rm = TRUE), 2),
                mean = ~round(mean(., na.rm = TRUE), 2),
                median = ~round(median(., na.rm = TRUE), 2),
                max = ~round(max(., na.rm = TRUE), 2),
                sd = ~round(sd(., na.rm = TRUE), 2)))
  )
players_summary


In [ ]:
#selecting necessary variables

players_select <- players |>
    select(subscribe, experience, played_hours, Age)

#converting categorical variables to factors

players_select <- players_select |>
    mutate(subscribe = as.factor(subscribe), experience = as.factor(experience))

#removing rows with missing vals

players_clean <- players_select |>
    filter(!is.na(played_hours), !is.na(Age))

#mean values for numeric variables

mean_summary <- players_clean |>
    summarise(mean_played_hours = mean(played_hours, na.rm = TRUE), mean_age = mean(Age, na.rm = TRUE)) |>
    mutate(across(everything(), ~round(.x, 2)))

head(players_clean)
mean_summary

## **Data Visualization**

In [ ]:
#Plot 1: Categorical Plot - Experience vs. Subscription Status
options(repr.plot.width = 12, repr.plot.height = 8)

cat_plot <- players_clean |>
    group_by(experience, subscribe) |>
    summarise(count = n(), .groups = "drop") |>
    group_by(experience) |>
    mutate(prop = count / sum(count)) |>
    filter(subscribe == "TRUE") |>
    ggplot(aes(x = experience, y = prop, fill = experience)) +
    scale_y_continuous(labels = scales::percent_format()) +
    geom_col(show.legend = FALSE) + 
    labs(title = "Newsletter Subscription Rate by Experience level", x = "Experience Level", y = "Subscription Rate (%)") + 
    theme(text = element_text(size = 20)) 
cat_plot

In [ ]:
#Plot 2: Numeric Plot 1 - Age vs. Subscription Status

options(repr.plot.width = 12, repr.plot.height = 8)

num_plot1 <- players_clean |>
    ggplot(aes(x = subscribe, y = Age, color = subscribe)) +
    geom_boxplot(outlier.shape = NA, alpha = 0.4) +
    geom_jitter(width = 0.2, alpha = 0.5, size = 1) +
    labs(x = "Subscription Status", y = "Age", title = "Age vs. Subscription Status") +
    theme(text = element_text(size = 20)) + 
    theme(legend.position = "none")

num_plot1

In [ ]:
#Plot 3: Numeric Plot 2 - Played Hours vs. Subscription Status

num_plot2 <- players_clean |>
    ggplot(aes(x = subscribe, y = played_hours, color = subscribe)) +
    geom_boxplot(outlier.shape = NA, alpha = 0.4) +
    geom_jitter(width = 0.2, alpha = 0.5, size = 1) +
    scale_y_continuous(trans = "log1p") +
    labs(x = "Subscription Status", y = "Played Hours", title = "Played Hours (log scale) vs. Subscription Status") +
    theme(text = element_text(size = 20))  +
    theme(legend.position = "none")

num_plot2

## **Visualization Insights:**

**Plot 1** 
- Veteran players subscribed the least (68%), wheareas regular players subscribed the most (81%)
- All other experience levels subscribed near similar levels (70-77%)
  
**Plot 2**
- The median age of players subscribed were below the age of 20, whereas the median age of players that did not subscribe were above the age of 20
- The spread of players that did not subscribe to te newletter is larger than the spread of players that subscribed

**Plot 3**
- The logged median played hours of subscribed players is higher than the median played hours of players who were not subscribed
- The spread of played hours for subscribed players is larger than the spread of played hours for players who were not subscribed

## **Modelling**

In [ ]:
set.seed(123)

players <- players_clean |>
                    drop_na() |>
                    select(experience, subscribe, Age, played_hours) |>
                    mutate(experience = as_factor(experience) |>
                                           fct_relevel("Beginner", "Amateur", "Regular", "Pro", "Veteran"),
                          subscribe = as_factor(subscribe),
                          Age = as.integer(Age))

players_resampled <- players |>
                        mutate(experience = as.integer(experience)) |>
                        rep_sample_n(size = 194, replace = TRUE, reps = 10)

subscribers_split <- players_resampled |>
                        initial_split(prop = 0.75)

subscribers_training <- training(subscribers_split)
subscribers_testing <- testing(subscribers_split)

subscribers_vfold <- vfold_cv(subscribers_training, v = 5, repeats = 5, strata = subscribe)

subscribers_k_values <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

knn_classification_spec <- nearest_neighbor(weight = "rectangular", neighbors = tune()) |>
                            set_engine("kknn") |>
                            set_mode("classification")

subscribers_recipe <- recipe(subscribe ~ experience + played_hours + Age, data = subscribers_training) |>
                                step_smote(subscribe) |>
                                step_zv() |>
                                step_normalize(all_predictors())

subscribers_results <- workflow() |>
                        add_recipe(subscribers_recipe) |>
                        add_model(knn_classification_spec) |>
                        tune_grid(resamples = subscribers_vfold, grid = subscribers_k_values) |>
                        collect_metrics() |>
                        filter(.metric == "accuracy")

subscribers_results_plot <- subscribers_results |>
                                ggplot(aes(x = neighbors, y = mean)) +
                                    geom_point() +
                                    geom_line() +
                                    ggtitle("Accuracy vs Neighbors") +
                                    xlab("Neighbors") +
                                    ylab("Accuracy")
subscribers_results_plot

In [ ]:
subscribers_best_k <- subscribers_results |>
    arrange(desc(mean)) |>
    slice(1) |>
    pull(neighbors)

"Best k-value: "
subscribers_best_k

best_knn_spec <- nearest_neighbor(weight = "rectangular", neighbors = subscribers_best_k) |>
    set_engine("kknn") |>
    set_mode("classification")

subscribers_fit <- workflow() |>
    add_recipe(subscribers_recipe) |>
    add_model(best_knn_spec) |>
    fit(data = subscribers_training)

In [ ]:
subscribers_predictions <- predict(subscribers_fit, subscribers_testing) |>
    bind_cols(subscribers_testing)

subscribers_acc <- subscribers_predictions |>
    metrics(truth = subscribe, estimate = .pred_class) |>
    filter(.metric == "accuracy") |>
    pull(.estimate)

subscribers_prec <- subscribers_predictions |>
    precision(truth = subscribe, estimate = .pred_class, event_level = "second") |>
    pull(.estimate)

subscribers_rec <- subscribers_predictions |>
    recall(truth = subscribe, estimate = .pred_class, event_level = "second") |>
    pull(.estimate)

subscribers_conf_mat <- subscribers_predictions |>
    conf_mat(truth = subscribe, estimate = .pred_class)

cat("Accuracy: ", subscribers_acc, "\n")
cat("Precision: ", subscribers_prec, "\n")
cat("Recall: ", subscribers_rec, "\n")
subscribers_conf_mat


I used the KNN-model for this 

## **Discussion**

A total of 196 players were recorded in this study on the UBC Minecraft research server led by Prof. Frank Wood in an open-access environment where player interactions are automatically logged. This study was conducted for a bit more than 2 months, from 2024-08-01 to 2024-11-30. Of the players who did subscribe to a game related newsletter, we found that subscription rates differed by experience, age, and played hours. In terms of experience level, players who were veterans at the game were found to have the lowest subscription rate (68%) and regular players were found to have the highest subscription rate (81%), all other experience levels subscribed near similar levels (70-77%). The median age of subscribed players was below 20, whereas the median age of unsubscribed players was above 20. The logged median played hours of subscribed players was higher than the logged median played hours for unsubscribed players, the latter also has a smaller spread of datapoints.

When using the KNN-model to classify data under these 3 variables, the accurage, precision, and recall were all near 90% showing that these 3 factors are good predictors for subsription status. 

The results of the study were expected. Younger ages, regular experience levels, and higher played hours were linked to higher subscription rates. This could be because younger players have a tendence to spend more time in the game. Regular experience levels indicate that a player has played the game more than a beginner, but lacks the experience and knowledge of a pro or veteran that has already spent a lot of time on the game, so subsribing to a game-related newsletter would seem more attractive to players seeking to learn more or progress in the game. Younger players could be more interested in subscribing to a game related newsletter because they have more time than older players. Players with higher played hours could be more interested in subscribing to a game-related newsletter because they spend more time and are more attached to the game. To accomplish the goal of figuring out the influences that impact subscription rate to the game-related newsletter to target players to subscribe, more studies must be conducted to understand why player's age, experience level, and played hours impacts their subscription rate. 

Some limitations in this dataset are that most players in the study spent less than hour on Minecraft over the course of the study. There were also many outliers in the dataset. Having players with a wider range of playtime, or having a larger dataset, would provide better understanding to player subscription behaviour.


**References**